In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

# Set up GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is available")
    except RuntimeError as e:
        print(e)
else:
    print("GPU is not available, using CPU")

base_path = "/kaggle/input/arabic-egy-asr-16k"
train_wavs_path = os.path.join(base_path, "train")
adapt_wavs_path = os.path.join(base_path, "adapt")
train_metadata_path = os.path.join(base_path, "train.csv")
adapt_metadata_path = os.path.join(base_path, "adapt.csv")

2024-07-03 17:28:07.154021: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 17:28:07.154128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 17:28:07.288112: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU is available


In [2]:
def extract_features(file_path, sr=16000, n_mfcc=13):
    try:
        audio, _ = librosa.load(file_path, sr=sr)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
        mfccs = mfccs.T
        return mfccs
    except Exception as e:
        print(f"Error in extract_features for file {file_path}: {str(e)}")
        return None

In [3]:
def load_data(metadata_path, wavs_path, max_transcript_length=280):
    df = pd.read_csv(metadata_path)
    X = []
    y = []
    max_len = 0
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Loading data"):
        file_name = row['audio']
        file_path = os.path.join(wavs_path, file_name + '.wav')

        if pd.isna(row['transcript']) or (isinstance(row['transcript'], str) and len(row['transcript']) > max_transcript_length):
            continue

        if not os.path.exists(file_path):
            print(f"Warning: File not found: {file_path}")
            continue

        try:
            features = extract_features(file_path)

            if features is None or np.isnan(features).any():
                print(f"Warning: Invalid features for file: {file_path}")
                continue

            max_len = max(max_len, features.shape[0])
            X.append(features)
            y.append(row['transcript'])
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")

    return X, y, max_len

In [4]:
def pad_features(features, max_len):
    if features.shape[0] < max_len:
        return np.pad(features, ((0, max_len - features.shape[0]), (0, 0)), mode='constant')
    else:
        return features[:max_len, :]

In [5]:
# Load and preprocess data
print("Loading training data...")
X_train, y_train, train_max_len = load_data(train_metadata_path, train_wavs_path, max_transcript_length=280)
print("Loading adaptation data...")
X_adapt, y_adapt, adapt_max_len = load_data(adapt_metadata_path, adapt_wavs_path, max_transcript_length=280)


Loading training data...


Loading data: 100%|██████████| 50715/50715 [27:30<00:00, 30.74it/s]


Loading adaptation data...


Loading data: 100%|██████████| 2199/2199 [01:03<00:00, 34.74it/s]


In [7]:
max_len = max(train_max_len, adapt_max_len)
print(f"Maximum sequence length: {max_len}")

# Pad sequences
X_train = [pad_features(x, max_len) for x in X_train]
X_adapt = [pad_features(x, max_len) for x in X_adapt]

# Use all adapt data for validation
X_val = X_adapt
y_val = y_adapt

# Convert to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_val: {X_val.shape}")
print(f"Number of training samples: {len(y_train)}")
print(f"Number of validation samples: {len(y_val)}")


Maximum sequence length: 1082
Shape of X_train: (50709, 1082, 13)
Shape of X_val: (2190, 1082, 13)
Number of training samples: 50709
Number of validation samples: 2190


In [8]:
# Create character mappings
chars = set(''.join(y_train + y_val))
char_to_index = {char: i for i, char in enumerate(sorted(chars))}
index_to_char = {i: char for char, i in char_to_index.items()}
vocab_size = len(char_to_index)

max_transcript_len = max(len(t) for t in y_train + y_val)


In [10]:
def ctc_loss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.reduce_sum(tf.cast(tf.not_equal(y_true, -1), tf.int64), axis=-1)

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = tf.expand_dims(label_length, axis=-1)

    # Ensure input_length is always greater than or equal to label_length
    input_length = tf.maximum(input_length, label_length + 1)

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [11]:
import os
from tensorflow.keras.models import load_model

# Load the acoustic model
base_path = "/kaggle/input/mtc-train" 

am_load_path = os.path.join(base_path, 'acoustic_model_train_1.h5')
am = load_model(am_load_path, custom_objects={'ctc_loss': ctc_loss})
print(f"Acoustic model loaded from {am_load_path}")

# Load the language model
lm_load_path = os.path.join(base_path, 'language_model_train_1.h5')
lm = load_model(lm_load_path)
print(f"Language model loaded from {lm_load_path}")

Acoustic model loaded from /kaggle/input/mtc-train/acoustic_model_train_1.h5
Language model loaded from /kaggle/input/mtc-train/language_model_train_1.h5


In [12]:
base_path = "/kaggle/input/mtc-train" 

# Load the acoustic model
am_load_path = os.path.join(base_path, 'acoustic_model_train_2.h5')
am = load_model(am_load_path, custom_objects={'ctc_loss': ctc_loss})
print(f"Acoustic model loaded from {am_load_path}")

# Load the language model
lm_load_path = os.path.join(base_path, 'language_model_train_2.h5')
lm = load_model(lm_load_path)
print(f"Language model loaded from {lm_load_path}")

Acoustic model loaded from /kaggle/input/mtc-train/acoustic_model_train_2.h5
Language model loaded from /kaggle/input/mtc-train/language_model_train_2.h5


In [14]:
def reformat_lexicon(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', encoding='utf-8') as outfile:
        for line in infile:
            parts = line.strip().split()
            if len(parts) >= 2:
                word = parts[0]
                pronunciation = ' '.join(parts[1:])
                outfile.write(f"{word}\t{pronunciation}\n")
            elif len(parts) == 1 and parts[0]:  # Handle single-word lines
                word = parts[0]
                outfile.write(f"{word}\t{word}\n")  # Use the word as its own pronunciation
            else:
                print(f"Skipping empty line")

# Paths
input_lexicon_path = "/kaggle/input/croups-2/words (1).txt"
output_lexicon_path = "/kaggle/working/final_formatted.txt"

# Reformat the lexicon
reformat_lexicon(input_lexicon_path, output_lexicon_path)

print(f"Reformatted lexicon saved to: {output_lexicon_path}")

# Display the first few lines of the reformatted lexicon
print("\nFirst few lines of the reformatted lexicon:")
with open(output_lexicon_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 10:  # Display first 10 lines
            print(line.strip())
        else:
            break

# Count the number of entries in the reformatted lexicon
with open(output_lexicon_path, 'r', encoding='utf-8') as f:
    entry_count = sum(1 for line in f if line.strip())
print(f"\nTotal entries in the reformatted lexicon: {entry_count}")

Reformatted lexicon saved to: /kaggle/working/final_formatted.txt

First few lines of the reformatted lexicon:
على	على
إنها	إنها
عار	عار
في	في
الوقت	الوقت
اللي	اللي
كانت	كانت
بتتعامل	بتتعامل
مع	مع
أخويا	أخويا

Total entries in the reformatted lexicon: 69062


In [15]:
!pip install jiwer

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.3 MB/s eta 0:00:00a 0:00:01


In [16]:
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from jiwer import wer
from tqdm import tqdm

# Function to calculate WER
def calculate_wer(reference, hypothesis):
    return wer(reference, hypothesis)

# Load lexicon (assuming you have a lexicon file)
def load_lexicon(lexicon_path):
    lexicon = {}
    with open(lexicon_path, 'r', encoding='utf-8') as f:
        for line in f:
            word, pronunciation = line.strip().split('\t')
            lexicon[word] = pronunciation
    return lexicon

# Assuming you have a lexicon file, load it
lexicon_path = "/kaggle/working/final_formatted.txt" 
lexicon = load_lexicon(lexicon_path)

# Function to improve prediction using lexicon
def improve_prediction(text, lexicon):
    words = text.split()
    improved_words = []
    for word in words:
        if word in lexicon:
            improved_words.append(lexicon[word])
        else:
            improved_words.append(word)
    return ' '.join(improved_words)

# Function to decode CTC output
def decode_ctc(prediction):
    decoded = tf.keras.backend.ctc_decode(prediction, input_length=[prediction.shape[1]]*prediction.shape[0], greedy=True)[0][0]
    return decoded.numpy()

# Function to convert indices to text
def indices_to_text(indices):
    return ''.join([index_to_char[i] for i in indices if i != -1])

# Function to predict single audio
def predict_single_audio(file_path):
    features = extract_features(file_path)
    features = pad_features(features, max_len)
    features = np.expand_dims(features, axis=0)
    
    am_prediction = am.predict(features)
    decoded = decode_ctc(am_prediction)
    text = indices_to_text(decoded[0])
    
    # Improve prediction using lexicon
    improved_text = improve_prediction(text, lexicon)
    
    return improved_text

# Function to create prediction file
def create_prediction_file(test_dir, output_path):
    predictions = []
    audio_files = []
    
    # Iterate through all wav files in the test directory
    for filename in tqdm(os.listdir(test_dir), desc="Predicting"):
        if filename.endswith('.wav'):
            file_path = os.path.join(test_dir, filename)
            prediction = predict_single_audio(file_path)
            predictions.append(prediction)
            audio_files.append(filename)
    
    # Create a DataFrame with predictions
    results_df = pd.DataFrame({
        'audio': audio_files,
        'transcript': predictions
    })
    
    # Save predictions to CSV
    results_df.to_csv(output_path, index=False)
    print(f"Predictions saved to {output_path}")
    
    # Print statistics
    print(f"Total files processed: {len(audio_files)}")

# Predict single audio
single_audio_path = "/kaggle/input/mtc-test-5/test/test_sample_1000_clean.wav"  
prediction = predict_single_audio(single_audio_path)
print(f"Prediction for single audio: {prediction}")

# Create prediction file for test data
test_path = "/kaggle/input/mtc-test-5/test"
output_path = "/kaggle/working/predictions_m.csv"
create_prediction_file(test_path, output_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
Prediction for single audio: وتكتورطب عنا دقىفج


Predicting:   0%|          | 0/1726 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step


Predicting:   0%|          | 1/1726 [00:00<07:01,  4.09it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step


Predicting: 100%|█████████▉| 1725/1726 [06:30<00:00,  4.49it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


Predicting: 100%|██████████| 1726/1726 [06:30<00:00,  4.42it/s]

Predictions saved to /kaggle/working/predictions_m.csv
Total files processed: 1726
